# This notebook is used to compare the output of Michaels old scripts with the new output of bonndit

First we compute the results with michis scripts and the bonndit script

In [62]:
import numpy as np

In [4]:
DATA_DIR = '/home/olivier/Devel/test/samuel/in4michi/'

In [25]:
%%time
%%bash --out output --err error
source /home/olivier/Devel/venvs/bonndit/bin/activate
shore-response.py /home/olivier/Devel/test/samuel/in4michi/complete_data/ /home/olivier/Devel/test/samuel/in4michi/michi_result/response

CPU times: user 8.66 ms, sys: 0 ns, total: 8.66 ms
Wall time: 1min 3s


In [32]:
%%time
%%bash --out output --err error
source /home/olivier/Devel/venvs/bonndit/bin/activate
shore-deconvolve.py /home/olivier/Devel/test/samuel/in4michi/data.nii.gz \
                    /home/olivier/Devel/test/samuel/in4michi/bonndit_result/response.npz \
                    /home/olivier/Devel/test/samuel/in4michi/michi_result/odfnoconstraint.nrrd \
                    --pos none \
                    --csf /home/olivier/Devel/test/samuel/in4michi/michi_result/csfnoconstraint.nrrd \
                    --gm /home/olivier/Devel/test/samuel/in4michi/michi_result/gmnoconstraint.nrrd \
                    --wm /home/olivier/Devel/test/samuel/in4michi/michi_result/wmnoconstraint.nrrd \

CPU times: user 1.13 ms, sys: 4.56 ms, total: 5.69 ms
Wall time: 704 ms


In [19]:
print(output)

In [29]:
print(error)

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/numpy/linalg/linalg.py:1543: RuntimeWarning: divide by zero encountered in true_divide
  return s[..., 0]/s[..., -1]



In [6]:
%%time
%%bash --out output1 --err error1
source /home/olivier/Devel/venvs/bonndit/bin/activate
get_fodfs.py -i /home/olivier/Devel/test/samuel/in4michi/ -o /home/olivier/Devel/test/samuel/in4michi/bonndit_result/ -v

CPU times: user 2.76 ms, sys: 2.76 ms, total: 5.53 ms
Wall time: 826 ms


In [7]:
print(output1)

In [8]:
print(error1)

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
100%|██████████| 27/27 [00:00<00:00, 114621.67it/s]
Traceback (most recent call last):
  File "/home/olivier/Devel/venvs/bonndit/bin/get_fodfs.py", line 6, in <module>
    exec(compile(open(__file__).read(), __file__, 'exec'))
  File "/home/olivier/Devel/bonndit/scripts/get_fodfs.py", line 91, in <module>
    main()
  File "/home/olivier/Devel/bonndit/scripts/get_fodfs.py", line 81, in main
    dti_fa, dti_vecs, fawm, verbose=verbose)
  File "/home/olivier/Devel/bonndit/src/bonndit/shore.py", line 54, in fit
    shore_coeff = self._get_response(data, mask_csf, verbose)
  File "/home/olivier/Devel/bonndit/src/bonndit/shore.py", line 118, in _get_response
    return 

Now we load the response functions and check how much they differ.

In [63]:
michi_response = np.load(DATA_DIR + 'michi_result/response.npz')
michi_signal_csf = michi_response['csf']
michi_signal_gm = michi_response['gm']
michi_signal_wm = michi_response['wm']

In [64]:
response = np.load(DATA_DIR + 'bonndit_result/response.npz')
signal_csf = response['csf']
signal_gm = response['gm']
signal_wm = response['wm']

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
(michi_signal_csf-signal_csf)/michi_signal_csf

In [ ]:
(michi_signal_gm-signal_gm)/michi_signal_gm

In [ ]:
(michi_signal_wm-signal_wm)/michi_signal_wm

Now we load the calculated ODFs and compare them

In [65]:
import nrrd

In [74]:
michi_odf, options = nrrd.read(DATA_DIR + 'michi_result/odf.nrrd')
michi_odf2, options = nrrd.read(DATA_DIR + 'michi_result/odf2.nrrd')
bonndit_odf, options2 = nrrd.read(DATA_DIR + 'bonndit_result/odf_michi.nrrd')

In [19]:
michi_odf.shape

(16, 3, 3, 3)

In [22]:
bonndit_odf.shape

(16, 3, 3, 3)

In [75]:
(michi_odf[3] - bonndit_odf[3])/ michi_odf[3]

array([[[ 1.0977648e+00,  9.8660207e-01,  2.4256656e+00],
        [-1.4154944e+00,  5.7278299e-01, -4.6633720e-01],
        [-7.2049398e+00,  2.1017464e-01,  9.5323539e-01]],

       [[ 1.6375872e-01,  1.0284528e+00, -1.5610442e-02],
        [-6.4690791e-02, -1.7208376e+01,  1.6342794e+00],
        [ 9.3855399e-01, -5.0661693e+00, -2.8632977e+00]],

       [[ 2.1025751e+00,  2.3027145e+01,  9.9796791e+00],
        [ 3.5082271e+00,  1.5874412e+00, -2.1131596e-01],
        [ 3.1784301e+00, -4.0044469e-01,  1.1185819e+00]]], dtype=float32)